In [ ]:
import numpy as np
import pandas as pd
!pip install datasets
from datasets import load_dataset
import tensorflow as tf

In [ ]:
# Load civil_comments dataset
dataset = load_dataset("google/civil_comments")
print(dataset['train'][0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'text': "This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!", 'toxicity': 0.0, 'severe_toxicity': 0.0, 'obscene': 0.0, 'threat': 0.0, 'insult': 0.0, 'identity_attack': 0.0, 'sexual_explicit': 0.0}


In [ ]:
df = pd.DataFrame(dataset['train'])
df.head()

,text,toxicity,severe_toxicity,obscene,threat,insult,identity_attack,sexual_explicit
0,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0
1,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0
2,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0
3,Is this something I'll be able to install on m...,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0
4,haha you guys are a bunch of losers.,0.893617,0.021277,0.0,0.0,0.87234,0.021277,0.0


In [ ]:
df.iloc[200]

,200
text,To meet these people with threats of violence ...
toxicity,0.2
severe_toxicity,0.0
obscene,0.0
threat,0.0
insult,0.0
identity_attack,0.0
sexual_explicit,0.0


In [ ]:
threshold = 0.5
class_columns = ['toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit']

(df[class_columns] >= threshold).sum()

,0
toxicity,144334
severe_toxicity,13
obscene,9603
threat,4280
insult,106534
identity_attack,13410
sexual_explicit,4686


In [ ]:
severe_toxicity_counts = (df['severe_toxicity'] > 0.1).sum()
print(severe_toxicity_counts)

50978


In [ ]:
severe_toxicity_counts = (df['severe_toxicity'] > 0.5).sum()
print(severe_toxicity_counts)

8


In [ ]:
toxicity_counts = (df['toxicity'] > 0.5).sum()
print(toxicity_counts)

106438


In [ ]:
toxicity_counts = (df['toxicity'] > 0.7).sum()
print(toxicity_counts)

45451


In [ ]:
toxicity_counts = (df['toxicity'] > 0.2).sum()
print(toxicity_counts)

377686


During the exploration of the dataset, we discovered that the labels for each class
(e.g., toxicity, severe_toxicity, etc.) are not binary (0 or 1), but rather probabilistic values
ranging between 0 and 1. This means the dataset uses soft labels, indicating the likelihood
that a given comment belongs to each class rather than hard classification.

For example, a 'severe_toxicity' label might have a value like 0.2 or 0.6, instead of just 0 or 1.
Training the model using these soft labels can be more appropriate in such cases,
as it helps the model learn more nuanced patterns in the data.

Therefore, in the next step, we will train the model using these probability values as targets
instead of thresholding or converting them to binary values.
This approach should help the model capture the inherent uncertainty in the labeling process.

After analyzing the poor performance of the model in the Toxicity2 notebook, I investigated potential issues related to the text preprocessing pipeline. I found that using a more modern and integrated preprocessing approach such as `TextVectorization` from TensorFlow can lead to better results compared to traditional tokenization with manual preprocessing. The `TextVectorization` layer allows for built-in standardization, tokenization, vocabulary creation, and optional stop word filtering, all within the model pipeline, which improves consistency and performance.

Furthermore, I realized that removing stop words might actually harm the model's understanding of the text in this particular context. In toxicity detection tasks, stop words such as "you", "are", or "not" can carry significant semantic weight and contribute to the tone and intent of a sentence. Removing them might lead to a loss of important contextual or syntactic information.

Based on these observations and best practices in modern NLP, I will now use the `TextVectorization` layer without explicitly removing stop words, to preserve the natural structure of the sentences and allow the model to learn from the full linguistic context.

Reference:  
TensorFlow TextVectorization documentation – https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization


In [ ]:
from tensorflow.keras.layers import TextVectorization

comment_text = df['text']
lables = df[df.columns[2:]].values

print(comment_text)
print(lables)

0          This is so cool. It's like, 'would you want yo...
1          Thank you!! This would make my life a lot less...
2          This is such an urgent design problem; kudos t...
3          Is this something I'll be able to install on m...
4                       haha you guys are a bunch of losers.
                                 ...                        
1804869    Maybe the tax on "things" would be collected w...
1804870    What do you call people who STILL think the di...
1804871    thank you ,,,right or wrong,,, i am following ...
1804872    Anyone who is quoted as having the following e...
1804873    Students defined as EBD are legally just as di...
Name: text, Length: 1804874, dtype: object
[[0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 ...
 [0.         0.         0.         0.         0.         0.        ]
 [0.0

In [ ]:
#vectorizer = TextVectorization(max_tokens=10000, output_sequence_length=1800, output_mode='int')
#vectorizer.adapt(comment_text.values)
#vectorized_text = vectorizer(comment_text.values)

In [ ]:
print(len(df))

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Apply cleaning
df = pd.DataFrame(dataset['train'])
df['text'] = df['text'].map(lambda x: clean_text(x))